# Data Structures & Algorithms

This page contains a compilation of Data Structures & Algorithms I have learnt based on my research. Some data structures or algorithms may be drawn from the courses I have taken (CS2030 - Programming Methodology II or CS2040 - Data Structures and Algorithms), but they will only be **a part of the algorithm** which I will be documenting and **not** the actual algorithm itself.

## Shunting Yard Algorithm (and Post-fix Calculator)

- The Shunting Yard algorithm converts an infix mathematical expression into a postfix equivalent, abiding by the **PEMDAS** rule (Order of Operation). It will then use the postfix calculator to evaluate the values.

| Infix                             | Postfix                             |
| ------                            | -----                               |
| (2 + 6) * 3                       | 2 6 + 3 *                           |
| - 10 / (20 / 2^2 * 5 / 5) * 8 - 2 | 10 20 2 2 ^ / 5 * 5 / / 8 * - 2.0 - |


**PEMDAS**
- **P**arenthesis
- **E**xponent
- **M**ultiplication
- **D**ivision
- **A**ddition
- **S**ubtraction

### The algorithm

- There will be a the original expression using a queue (east), stack (south) and a processed postfix expression (west)
    - For a more detailed explanation on stacks and queues, please refer to my <a href="./Python 3 Crash Course 2.ipynb">Python 3 Crash Course 2</a>, on the Anagram problem for CS1010E
    
- Evaluate the expression from left to right
- Enqueue all numbers immediately
- Push open brackets `(` into the stack (south)
    - If a close bracket is found, pop from stack until open bracket is found
    - Do not enqueue the open bracket
- Operators
    - Output from stack if it has greater or equal precedence or until we hit an open bracket
- Keep enqueuing the top most item from the stack (south) into the queue (west)
- Use the postfix calculator algorithm to evaluate the final value


### How to use this code

- Convert the following operators in your expression

| From     | To         |
| ---      | ---        |
| 3(3 + 2) | 3 * (3 + 2)|
| Exponent | ^          |
| x        | *          |
| ÷        | /          |

- Run the code `Shunting_Yard.calculate("math expression")`

In [1]:
import re

precedence = {"+" : 0, "-" : 0, "*" : 1, "/" : 1, "^" : 2}

class Shunting_Yard:    
    
    @staticmethod
    def higher_precedence(first, second):
        global precedence
        return precedence[first] <= precedence[second]

    @staticmethod
    def calculate(expression):
        expression = expression.replace("{","(").replace("[", "(").replace("}",")").replace("]",")")
        
        east = re.findall("[+/*\-()^]|\d*\.\d+|\d+", expression) # \d*\.\d+|\d+ - find all decimals and integers
        
        # Check for negative signs in the center ["*", "-", "3"] becomes ["*", "-3"]
        for i in range(len(east)):
            if east[i] == "-":
                if i - 1 >= 0 and east[i - 1] in ["+", "-", "*", "/", "^", "("]:
                    east[i] = ""
                    east[i + 1] = str(0 - float(east[i + 1]))
                    
        east = [x for x in filter(lambda x : x != "", east)]
        
        west = [] # A queue
        south = [] # A stack

        for char in east:
            if re.match("-?\d+(\.\d+)?",char):
                west.append(float(char))
            elif char == ")":
                while south and south[-1] != "(":
                    west.append(south.pop())
                south.pop() # Remove the (
            elif char == "(":
                south.append(char)
            else:
                while south and south[-1] != "(" and Shunting_Yard.higher_precedence(char, south[-1]):
                    west.append(south.pop())
                south.append(char)

        while south: 
            west.append(south.pop())
            
            
        print(west)

        return Shunting_Yard.postfix_calculator(west)
    
    @staticmethod
    def postfix_calculator(expression):
        global numbers
        
        east = expression
        
        south = []
        
        while east:
            if not re.match("-?\d+(\.\d+)?",str(east[0])) :
                operator = east.pop(0)
                
                if len(south) < 2 and operator == "-": # Removes the -ve sign at the start
                    num1 = south.pop()
                    south.append(Shunting_Yard.evaluate(operator, 0, num1))
                else:
                    num2 = south.pop()
                    num1 = south.pop()
                    south.append(Shunting_Yard.evaluate(operator, num1, num2))
            else:
                south.append(east.pop(0))
                
        return south.pop()
                
    @staticmethod
    def evaluate(operator, num1, num2):
        if operator == "^":
            return num1 ** num2
        elif operator == "*":
            return num1 * num2
        elif operator == "/":
            return num1 / num2
        elif operator == "+":
            return num1 + num2
        else:
            return num1 - num2

In [2]:
Shunting_Yard.calculate("4 * (1+2*(9/3)-5)")

[4.0, 1.0, 2.0, 9.0, 3.0, '/', '*', '+', 5.0, '-', '*']


8.0

In [3]:
Shunting_Yard.calculate("- 10 / (20 / 2^2 * 5 / 5) * 8 - 2")

[10.0, 20.0, 2.0, 2.0, '^', '/', 5.0, '*', 5.0, '/', '/', 8.0, '*', '-', 2.0, '-']


-18.0

In [4]:
Shunting_Yard.calculate("100 / (-20 * (-5))")

[100.0, -20.0, -5.0, '*', '/']


1.0

In [5]:
Shunting_Yard.calculate("1000 / (-5 * -2 * -10 * -10)")

[1000.0, -5.0, -2.0, '*', -10.0, '*', -10.0, '*', '/']


1.0

In [6]:
Shunting_Yard.calculate("18 - [6 - {4 - (8 - 6 +3)}]")

[18.0, 6.0, 4.0, 8.0, 6.0, '-', 3.0, '+', '-', '-', '-']


11.0

In [7]:
Shunting_Yard.calculate("4 * (10 + 15 / 5 * 4 - 2 * 2)")

[4.0, 10.0, 15.0, 5.0, '/', 4.0, '*', '+', 2.0, 2.0, '*', '-', '*']


72.0

In [8]:
Shunting_Yard.calculate("-2.5 / (0.5 * -0.5)")

[2.5, 0.5, -0.5, '*', '/', '-']


10.0

## Flyod's Tortoise and Hare Algorithm

### Constraints
- Given an array of size N + 1, where there are only N integers. There is always at least a duplicate number
- Only 1 duplicate integer
- Can be duplicated multiple times
- Must not modify the array
- Must only use constant O(1) space
- Runtime complexity must be < O(N<sup>2</sup>)

In [3]:
def findDuplicate(intlist):
    tortoise = intlist[0]
    hare = intlist[0]
    
    while True:
        tortoise = intlist[tortoise]
        hare = intlist[intlist[hare]]
        
        if tortoise == hare:
            break
    
    tortoise = intlist[0]
    
    while tortoise != hare:
        tortoise = intlist[tortoise]
        hare = intlist[hare]
        
    return hare

In [7]:
findDuplicate([1,3,4,2,2])

2

In [8]:
findDuplicate([3,1,3,4,2])

3